# 2. Interactive Health Dashboard with Real-World Data

Let's create a health dashboard using a publicly available dataset. For this example, we'll use a dataset from kaggle. 

Dataset: https://www.kaggle.com/datasets/ashudata/covid19dataset

**Key Components:**
- **Global Overview**: A line chart showing the trend of confirmed COVID-19 cases worldwide over time.
- **Country Comparison**: An interactive tab where users can select multiple countries to compare their COVID-19 case trends.
- **Deaths and Recoveries**: A line chart comparing global deaths and recoveries over time.
- **Active Cases**: An area chart showing the trend of active cases across the globe.

**Example: Health Dashboard on Covid-19 Dataset**

First, download the dataset from the Kaggle and save it as `COVID_Data.csv`. You can download such datasets from Kaggle.

here is the link to the:

[Kaggle Covid-19 Dataset](https://www.kaggle.com/datasets/ashudata/covid19dataset)

Now, let's create the dashboard:

In [5]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the COVID-19 dataset
df = pd.read_csv('COVID_Data.csv')

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.title = 'COVID-19 Health Dashboard'



# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Check the structure of the dataset
print(df.head())

# If necessary, aggregate data by country and date (depending on dataset structure)
country_df = df.groupby(['Country', 'Date']).sum().reset_index()

# Aggregating data for global overview
global_df = df.groupby('Date').sum().reset_index()


   Unnamed: 0      Country       Date  Confirmed  Death  newConfirmed  \
0           1  Afghanistan 2019-12-31          0      0             0   
1           2  Afghanistan 2020-01-01          0      0             0   
2           3  Afghanistan 2020-01-02          0      0             0   
3           4  Afghanistan 2020-01-03          0      0             0   
4           5  Afghanistan 2020-01-04          0      0             0   

   newDeath  
0         0  
1         0  
2         0  
3         0  
4         0  


In [6]:


# Define the layout with multiple tabs for different visualizations
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("COVID-19 Health Dashboard", className="text-center text-primary mb-4"), width=12)
    ]),
    
    dcc.Tabs([
        dcc.Tab(label='Global Overview', children=[
            dcc.Graph(id='global-overview-graph', 
                      figure=px.line(global_df, x='Date', y='Confirmed', 
                                     title='Global Confirmed Cases Over Time'))
        ]),
        dcc.Tab(label='Country Comparison', children=[
            html.Div([
                html.Label('Select Countries:'),
                dcc.Dropdown(id='country-dropdown', 
                             options=[{'label': country, 'value': country} for country in country_df['Country'].unique()],
                             value=['United States', 'India', 'Brazil'], multi=True)
            ]),
            dcc.Graph(id='country-comparison-graph')
        ]),
        dcc.Tab(label='Deaths Analysis', children=[
            dcc.Graph(id='deaths-analysis-graph', 
                      figure=px.line(country_df, x='Date', y='Death', color='Country', 
                                     title='Deaths Over Time'))
        ]),
        dcc.Tab(label='New Cases', children=[
            dcc.Graph(id='new-cases-graph', 
                      figure=px.bar(global_df, x='Date', y='newConfirmed', 
                                    title='Global New Confirmed Cases Over Time'))
        ]),
        dcc.Tab(label='New Deaths', children=[
            dcc.Graph(id='new-deaths-graph', 
                      figure=px.bar(global_df, x='Date', y='newDeath', 
                                    title='Global New Deaths Over Time'))
        ])
    ])
])

@app.callback(
    Output('country-comparison-graph', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_country_comparison(selected_countries):
    filtered_df = country_df[country_df['Country'].isin(selected_countries)]
    fig = px.line(filtered_df, x='Date', y='Confirmed', color='Country', 
                  title='Confirmed Cases Comparison')
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, port = 8053)

This dashboard gives a comprehensive overview of COVID-19 data, with interactive features that allow users to explore different aspects of the pandemic's impact globally.